# Extract MILC output
- Dec 28, 2021

## Description of columns in parse script : 



- Solve time =  2 x sum of all mass-solve times


- quda-total-time = solve-time + mg-setup + epsilon

- summed_time = quda_total_time + contractions + epsilon

- milc_total_time = summed_time + err

In [1]:
! hostname

jupyter-venkitesh


In [2]:
import os, glob
import subprocess as sp
import numpy as np
import shutil

import re

import pandas as pd

import matplotlib.pyplot as plt
from ipywidgets import *

import itertools

In [3]:
%matplotlib widget

In [4]:
top_dir='/gpfs/alpine/lgt104/proj-shared/ayyar/runs_frontier/runs_jan27_2023'

op_dict={
# 'cg_summit_144':'/gpfs/alpine/lgt104/proj-shared/ayyar/runs_summit/runs_june8_2023/cg_144/output-cg_144_summit.kpp',
'cg_crusher_144':'~/mg_output_files/output-cg_144_crusher.kpp',
# 'cg_summit_192':'/gpfs/alpine/lgt104/proj-shared/ayyar/runs_summit/runs_june8_2023/cg_192/output-cg_192_summit.kpp',
# 'cg_frontier_192':' ~/mg_output_files/output-cg_192_frontier.kpp',
# 'cg_frontier_192_split':'~/mg_output_files/output-cg_192_frontier_with_split.kpp',

# 'cg_frontier_144_split1':'~/mg_output_files/output-cg_144_frontier_with_split.kpp',
# 'cg_frontier_144_split2':'~/mg_output_files/output-cg_144_frontier_aug_2023.kpp',
    
    
'mg_frontier_144_split':'~/mg_output_files/output-full_144_frontier_aug_2023.kpp',

# 'summit_144_savingNN'   :'/gpfs/alpine/lgt104/proj-shared/ayyar/runs_summit/runs_june8_2023/run_144/output-full_1_savingNN.kpp',
    
# 'old_crusher_2_hprec':'/gpfs/alpine/lgt104/proj-shared/ayyar/runs_crusher/runs_feb2_2023_mpich8.1.12/output-full-2.kpp',
# 'crusher_latest_144_savingNN':'~/mg_output_files/output-full_1_savingNN.kpp',
# 'crusher_latest_144_readingNN':'~/mg_output_files/output-full_2_readingNN.kpp',
# 'frontier_144_savingNN':'~/mg_output_files/output-full_1_savingNN_144_frontier.kpp',

# 'summit_192': '/gpfs/alpine/lgt104/proj-shared/ayyar/runs_summit/runs_june8_2023/run_192/output-full-run-1.kpp',

# 'frontier_192': '~/mg_output_files/output-full_1_correct_ip.kpp',
}

# list(op_dict.keys())
# list(op_dict.values())

In [5]:
# cmd='./parse.sh %s mg '%(fname1)
# op=sp.check_output(cmd,shell=True,text=True)

## New way in python
# cmd=['./parse.sh',' '+fname1,'mg', ' > op0.out']
# print(cmd)
# op=sp.run(cmd,check=True,capture_output=True,text=True).stdout


In [6]:
## Parse output files

op_files=[]
for count,fl in enumerate(list(op_dict.values())):
    op_file='op%s.out'%(count)
    run_type='cg' if count<1 else 'mg'   ## Assume first few runs are cg, rest mg ##!
    print(fl,run_type)
    cmd='./parse.sh %s %s > %s'%(fl,run_type,op_file)
    op=sp.check_output(cmd,shell=True,text=True)
    
    ## Run bash script to parse output
    # cmd=['./parse.sh',fl,run_type,' > op%s.out'%(count+1)]
    # op=sp.run(cmd,check=True,capture_output=True,text=True)
    
    print(cmd)
    print(op)
    ## Append output file names to list
    op_files.append(op_file)

/gpfs/alpine/lgt104/proj-shared/ayyar/runs_summit/runs_june8_2023/cg_144/output-cg_144_summit.kpp cg
./parse.sh /gpfs/alpine/lgt104/proj-shared/ayyar/runs_summit/runs_june8_2023/cg_144/output-cg_144_summit.kpp cg > op0.out

~/mg_output_files/output-cg_144_crusher.kpp cg
./parse.sh ~/mg_output_files/output-cg_144_crusher.kpp cg > op1.out

~/mg_output_files/output-cg_144_frontier_with_split.kpp cg
./parse.sh ~/mg_output_files/output-cg_144_frontier_with_split.kpp cg > op2.out

~/mg_output_files/output-cg_144_frontier_aug_2023.kpp mg
./parse.sh ~/mg_output_files/output-cg_144_frontier_aug_2023.kpp mg > op3.out

~/mg_output_files/output-full_144_frontier_aug_2023.kpp mg
./parse.sh ~/mg_output_files/output-full_144_frontier_aug_2023.kpp mg > op4.out



In [7]:
names=list(op_dict.keys())

    
df_data=pd.DataFrame([])

count=0
for op_file, name in zip(op_files,names):
    print(op_file,name)
    with open(op_file,'r') as f:
        lst=f.readlines()

    print(lst)
    dict_values={}
    dict_values['name']=name
    # Add masses
    keys=lst[0].rstrip('\n').split(' ')
    vals=lst[1].rstrip('\n').split(' ')

    # Add timings
    keys+=lst[2].rstrip('\n').split(' ')
    vals+=lst[3].rstrip('\n').split(' ')

    # Add values to dictionary
    for k,v in zip(keys,vals):
        dict_values[k]=np.float64(v)

    # df_data=df_data.append(dict_values,ignore_index=True)
    df_data=pd.concat([df_data,pd.DataFrame(dict_values,index=[count])])
    count+=1

cols=['name', 'startup', 'loading', 'mg-setup',
       'make/save-src', 'mass-01', 'mass-02', 'mass-03', 'mass-04-10', 'mass-01-10',
       'mass-11-20', 'solve-time','contractions', 'quda-total-time','summed-time', 'error',
       'milc-total-time']

df_data=df_data[cols].copy()

op0.out cg_summit_144
['mass-01-10 mass-11-20\n', '6.4975873000000001e+02 9.3773073199999999e+01\n', 'startup loading mg-setup solve-time make/save-src contractions summed-time quda-total-time milc-total-time error\n', '8.766544e+01 3.419834e+02 0 1.4870636064000000e+03 2.021131e+02 1.869174e+02 2.3057429463999997e+03 1667.382 2.332431e+03 2.668805360000033e+01\n']
op1.out cg_crusher_144
['mass-01-10 mass-11-20\n', '3.9220819000000000e+02 5.5289390299999980e+01\n', 'startup loading mg-setup solve-time make/save-src contractions summed-time quda-total-time milc-total-time error\n', '3.729403e+01 6.951410e+02 0 8.9499516059999996e+02 2.436905e+01 3.297167e+02 1.9815159406000000e+03 979.632 1.999218e+03 1.770205940000005e+01\n']
op2.out cg_frontier_144_split1
['mass-01-10 mass-11-20\n', '5.2196420000000012e+02 5.9785005750000018e+01\n', 'startup loading mg-setup solve-time make/save-src contractions summed-time quda-total-time milc-total-time error\n', '2.999796e+01 6.964853e+02 0 1.16349

In [8]:
df_data

,name,startup,loading,mg-setup,make/save-src,mass-01,mass-02,mass-03,mass-04-10,mass-01-10,mass-11-20,solve-time,contractions,quda-total-time,summed-time,error,milc-total-time
0,cg_summit_144,87.66544,341.98340,0.0000,202.11310,NaN,NaN,NaN,NaN,649.75873,93.773073,1487.063606,186.9174,1667.382,2305.742946,26.688054,2332.431
1,cg_crusher_144,37.29403,695.14100,0.0000,24.36905,NaN,NaN,NaN,NaN,392.20819,55.289390,894.995161,329.7167,979.632,1981.515941,17.702059,1999.218
2,cg_frontier_144_split1,29.99796,696.48530,0.0000,22.03987,NaN,NaN,NaN,NaN,521.96420,59.785006,1163.498412,341.9548,1244.695,2253.976342,17.703658,2271.680
3,cg_frontier_144_split2,30.19691,31.26494,710.1907,22.03118,0.000000,0.000000,0.000000,524.342365,NaN,60.068671,1168.822073,101.9306,1269.544,2064.436403,-12.267403,2052.169
4,mg_frontier_144_split,35.08017,635.07840,4966.5800,21.59077,26.958162,21.891919,21.822319,118.695360,NaN,62.424405,503.584331,73.4263,5482.692,6235.339971,17.368029,6252.708




- Solve time =  2 x sum of all mass-solve times


- quda-total-time = solve-time + mg-setup + epsilon

- summed_time = quda_total_time + contractions + epsilon

- milc_total_time = summed_time + err




In [9]:
# df_data.iloc[[3,10,11]]
# df_data.iloc[[2,5,7,8,9]]
# df_data.iloc[[3,8,11]]

In [10]:
df=df_data.copy()
times_lst=['mass-01', 'mass-02', 'mass-03', 'mass-04-10',
       'mass-11-20', 'startup', 'loading', 'mg-setup', 'solve-time',
       'make/save-src', 'contractions', 'quda-total-time','summed-time', 'error',
       'milc-total-time']

# for col in times_lst:
    # df[col]=df[col]*df['gpu_factor']
    
df[:]

,name,startup,loading,mg-setup,make/save-src,mass-01,mass-02,mass-03,mass-04-10,mass-01-10,mass-11-20,solve-time,contractions,quda-total-time,summed-time,error,milc-total-time
0,cg_summit_144,87.66544,341.98340,0.0000,202.11310,NaN,NaN,NaN,NaN,649.75873,93.773073,1487.063606,186.9174,1667.382,2305.742946,26.688054,2332.431
1,cg_crusher_144,37.29403,695.14100,0.0000,24.36905,NaN,NaN,NaN,NaN,392.20819,55.289390,894.995161,329.7167,979.632,1981.515941,17.702059,1999.218
2,cg_frontier_144_split1,29.99796,696.48530,0.0000,22.03987,NaN,NaN,NaN,NaN,521.96420,59.785006,1163.498412,341.9548,1244.695,2253.976342,17.703658,2271.680
3,cg_frontier_144_split2,30.19691,31.26494,710.1907,22.03118,0.000000,0.000000,0.000000,524.342365,NaN,60.068671,1168.822073,101.9306,1269.544,2064.436403,-12.267403,2052.169
4,mg_frontier_144_split,35.08017,635.07840,4966.5800,21.59077,26.958162,21.891919,21.822319,118.695360,NaN,62.424405,503.584331,73.4263,5482.692,6235.339971,17.368029,6252.708


In [11]:
sub_list=['name','mg-setup','solve-time','milc-total-time']
df[sub_list]

,name,mg-setup,solve-time,milc-total-time
0,cg_summit_144,0.0000,1487.063606,2332.431
1,cg_crusher_144,0.0000,894.995161,1999.218
2,cg_frontier_144_split1,0.0000,1163.498412,2271.680
3,cg_frontier_144_split2,710.1907,1168.822073,2052.169
4,mg_frontier_144_split,4966.5800,503.584331,6252.708


In [ ]:
sub_list=['name','mg-setup','mass-01', 'mass-02', 'mass-03', 'mass-04-10','mass-01-10',
       'mass-11-20', 'solve-time','milc-total-time']
df[sub_list].fillna('-')



# 192
# df[sub_list].iloc[[3,10,11]]

# 144
# df[sub_list].fillna('-').iloc[[2,5,7,9]]


# 144 vs 192 frontier
# df[sub_list].fillna('-').iloc[[8,11]]


# cg comparison

df[sub_list].iloc[[2,3,4,5]][['name','mg-setup','mass-01-10','mass-11-20', 'solve-time','milc-total-time']]

## Plots

In [ ]:
y_options=['mass-01', 'mass-02', 'mass-03', 'mass-04-10', 'mass-11-20',
       'startup', 'loading', 'mg-setup', 'solve-time', 'make/save-src',
       'contractions', 'summed-time', 'quda-total-time', 'milc-total-time',
       'error', 'mass-01-10']


def f_compare_plot(df,y,x='name'):
    # df.plot(x=x,y=y,kind='line',marker='D')
    
    display(df[[x,y]])
    
    x=df.name.values
    y=df[y].values
    print(x,y)
    plt.figure()
    plt.plot(x,y,marker='*',linestyle='')
    # plt.plot(y,marker='*')
    # plt.legend()
    plt.show()
    
# df.plot(x='name',y='summed-time',kind='line',marker='*')
f_compare_plot(df,'summed-time','name')

In [ ]:
interact(f_compare_plot,df=fixed(df),y=y_options,x=fixed('name'))

In [ ]:
def f_plot_together(df,labels_lst,markers_lst): 
    plt.figure()
    x=df.name.values

    for label,marker in zip(labels_lst,markers_lst):
        y=df[label].values
        plt.plot(x,y,marker=marker,label=label)

    plt.legend()


In [ ]:
markers=['*','D','H','x','d','p','1','2','h','v']
labels=['startup', 'loading', 'mg-setup', 'solve-time', 'make/save-src',
       'contractions', 'summed-time', 'quda-total-time', 'milc-total-time',
       'error']

labels1=['mg-setup', 'solve-time','summed-time', 'quda-total-time', 'milc-total-time']

labels2=['startup', 'loading','contractions','make/save-src','error']

f_plot_together(df,labels1,markers)
f_plot_together(df,labels2,markers)


In [ ]:
pwd